In [1]:
from data.dataset import Dataset
from data.feature.coordinate import CoordinateFeaturizer
from data.dataloader import DataLoader
from methods.ridge_method import RidgeMethod

import numpy as np

In [2]:
featurizer = CoordinateFeaturizer()
#trimer_dataset = Dataset.from_file('xe3_50.xyz', 3, featurizer)
#energy_base = trimer_dataset[-1][1][0]
dimer_dataset = Dataset.from_file('dataset/xe2_50.xyz', 1, featurizer)
trimer_dataset = Dataset.from_file('dataset/xe3_50.xyz', 1, featurizer)
rand_trimer_dataset = Dataset.from_file('dataset/xe3_dataset_dft.xyz', 1, featurizer)

In [3]:
dimer_dataset.X = np.array([np.vstack([dimer_dataset.X[i], np.zeros(3)]) for i in range(len(dimer_dataset.X))])

In [4]:
n_dimer_train = 40
n_trimer_train = 40
n_rand_trimer_train = 5000

dimer_train, dimer_val = dimer_dataset.split(
    [list(range(n_dimer_train)),
     list(range(n_dimer_train, len(dimer_dataset)))])
trimer_train, trimer_val = trimer_dataset.split(
    [list(range(n_trimer_train)),
     list(range(n_trimer_train, len(dimer_dataset)))])
rand_trimer_train, rand_trimer_val = rand_trimer_dataset.split(
    [list(range(n_rand_trimer_train)), 
     list(range(n_rand_trimer_train, len(rand_trimer_dataset)))])
train = DataLoader([dimer_train, trimer_train, rand_trimer_train])
val = DataLoader([dimer_val, trimer_val, rand_trimer_val])

In [5]:
train.X = np.array([np.concatenate(coord) for coord in train.X])
val.X = np.array([np.concatenate(coord) for coord in val.X])

In [6]:
method = RidgeMethod(train)
method.train()
train_result = method.predict(train.X)
val_result = method.predict(val.X)

In [7]:
train_result

array([[-611448.60854848],
       [-611445.77769547],
       [-611442.86645171],
       ...,
       [-613812.00019228],
       [-613206.9638002 ],
       [-611781.78924696]])

# Train %RMSE

In [8]:
rmse = np.sqrt(np.mean((train_result[:40] - train.y[:40])**2)) / np.std(train.y[:40])

In [9]:
rmse

8160324.6935197245

In [10]:
rmse = np.sqrt(np.mean((train_result[40:80] - train.y[40:80])**2)) / np.std(train.y[40:80])

In [11]:
rmse

58925.83855497622

In [12]:
rmse = np.sqrt(np.mean((train_result[80:] - train.y[80:])**2)) / np.std(train.y[80:])

In [13]:
rmse

2719.3506724142553

# Val %RMSE

In [14]:
rmse = np.sqrt(np.mean((val_result[:10] - val.y[:10])**2)) / np.std(train.y[:40])

In [15]:
rmse

8153519.821596638

In [16]:
rmse = np.sqrt(np.mean((val_result[10:20] - val.y[10:20])**2)) / np.std(train.y[40:80])

In [17]:
rmse

152918.5096352947

In [18]:
rmse = np.sqrt(np.mean((train_result[20:] - train.y[20:])**2)) / np.std(train.y[80:])

In [19]:
rmse

11185.853947792113